In [7]:
import pandas as pd

In [8]:
df_train = pd.read_csv("sell_in_lag.csv")

In [9]:
df_train[f'tn_futuro'] = df_train.groupby('product_id')['tn_norm'].shift(-2)

In [10]:
df_train.to_csv('sell_in_train.csv', index=False)